### 0. 비교분석을 위해 LDA model 만들기.

In [1]:
import pandas as pd
import numpy as np
import pickle
from pprint import pprint
import re

In [2]:
with open("data/cleaned_data.pk", "rb") as f:
    data = pickle.load(f)

data.reset_index(drop=True, inplace=True)
print(data.head())
print(data.info())

             Date User                Message
0  2018. 5. 3. 00  주현                엥 결국 안먹음
1  2018. 5. 3. 00  주현    너 그 야구부 엠티는 가는거야 그래서
2  2018. 5. 3. 00  현영                      안가
3  2018. 5. 3. 00  현영        근엄마한테갈거라고말해놓긴햇는디
4  2018. 5. 3. 00  현영                  보내준다허면
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22914 entries, 0 to 22913
Data columns (total 3 columns):
Date       22914 non-null object
User       22912 non-null object
Message    22914 non-null object
dtypes: object(3)
memory usage: 537.2+ KB
None


### 2. 초록 없는 데이터 제거  및 분석시기 설정하기

In [3]:
# 시간정보 열 datetime 정보로 변환
data['Date'] = pd.to_datetime(data['Date'])
# 인덱스 넣기
data = data.set_index('Date')
data.head()

,User,Message
Date,,
2018-05-03,주현,엥 결국 안먹음
2018-05-03,주현,너 그 야구부 엠티는 가는거야 그래서
2018-05-03,현영,안가
2018-05-03,현영,근엄마한테갈거라고말해놓긴햇는디
2018-05-03,현영,보내준다허면


In [6]:
# LDA를 위해 2019년도 데이터만 불러오기.
year2019 = data["2019-01-01" : "2019-12-31"]
slice2 = list(year2019['Message'])

tokenized_data = [msg.split() for msg in slice2]
print(tokenized_data[:10])
print(len(tokenized_data))

[['얘두라', '따랑해', '새해복', '많이받아❤️☺️'], ['사랑해', '민나상', '모두들'], ['오메데또'], ['앗', '언니자화상이다'], ['졸귀탱'], ['야'], ['얘들아'], ['제발', '도와줘'], ['혹시', '니들도'], ['프로젝트', '3했니', '파일입출력']]
13858


### 3. LDA & Dynamic Topic Model 돌리기

In [7]:
from gensim.models import ldamodel
from gensim.models import ldaseqmodel
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary, bleicorpus
from gensim.matutils import hellinger
from gensim import corpora
from tqdm import tqdm_notebook
from time import time

import os

##### dictionary와 doc2bow 만들기 ( LDA에서 2019년도 것만 사용)

In [8]:
# Create Dictionary
if not os.path.exists('kakao(LDA)_dict'):
    dictionary = corpora.Dictionary(tokenized_data)
    #dictionary.filter_extremes(no_below=5, no_above=500)  # 이 줄의 코드는 n회 미만 또는 n회 이상을 지울 때 사용
    dictionary.save('kakao(LDA)_dict')
    print(dictionary)
else:
    dictionary = Dictionary.load('kakao(LDA)_dict')

# Term Document Frequency (convert tokenized documents into a Document-Term Matrix)    
if not os.path.exists('kakao(LDA)_corpus'):
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]
    corpora.BleiCorpus.serialize('kakao(LDA)_corpus', corpus)
else:
    corpus = bleicorpus.BleiCorpus('kakao(LDA)_corpus')

Dictionary(15576 unique tokens: ['따랑해', '많이받아❤️☺️', '새해복', '얘두라', '모두들']...)


##### Run LDA model 

In [9]:
# DTM 분석에서 best topic으로 나온 결과를 비교하기 위해 같은 토픽 수로 설정.
NUM_TOPICS = 8

if not os.path.exists('kakao(LDA)_model'):
    lda_model = ldamodel.LdaModel(corpus=corpus, id2word=dictionary,
                                 num_topics=NUM_TOPICS, passes=100)
    lda_model.save('kakao(LDA)_model')
else:
    lda_model = ldamodel.LdaModel.load('kakao(LDA)_model')

##### Run DTM Model

In [ ]:
# DTM 분석에서 best topic으로 나온 결과를 비교하기 위해 같은 토픽 수로 설정.
NUM_TOPICS = 8

dtm_model = ldaseqmodel.LdaSeqModel.load('kakao_dtm_model_8')

##### LDA와 DTM 결과 비교해보기

In [10]:
# 해당 토픽에 대한 결과를 직접 찍어봐서 얼마나 다른가 보는거야

lda_model.show_topic(topicid=0, topn=20) #show_topic이 예쁘게 나와

[('와', 0.026996456),
 ('지의', 0.010319685),
 ('슈발', 0.00938031),
 ('어제', 0.008536017),
 ('너무', 0.008114147),
 ('나는', 0.0075079184),
 ('내', 0.0067872196),
 ('개웃기네', 0.006454602),
 ('https', 0.005857338),
 ('데분개', 0.004585403),
 ('스발', 0.004251745),
 ('오키', 0.0034917565),
 ('휴', 0.0033159594),
 ('계속', 0.0029926023),
 ('딱', 0.0029020165),
 ('졸라', 0.0026146),
 ('하고', 0.0026077342),
 ('실습', 0.002589013),
 ('교수님이', 0.0025836434),
 ('쥑이네', 0.0024566702)]

In [ ]:
dtm_model.print_topic(topic=0, time=2, top_terms=20)

- 강사님 코드에선 패턴과 확률값 차이가 많이 났음
- DTM은 2018, 2019년도 모델을 학습을 하고 그 흐름이 파라미터가 반영돼서 넘어옴(데이터, 토픽의 분포가 반영돼서)
- 근데 LDA는 그냥 2019년도 데이터만 이용 

##### coherence score 계산 비교

In [11]:
# DTM coherence score 비교를 위해 데이터 불러오기.

dtm_corpus = corpus = bleicorpus.BleiCorpus('kakao(DTM)_corpus')
dtm_dictionary = Dictionary.load('kakao(DTM)_dict')
processing_data = [msg.split() for msg in data['Message']]

In [ ]:
# 각 모델별 coherence score 계산.
lda_cs = CoherenceModel(model=lda_model, texts=tokenized_data,
                        corpus=corpus, dictionary=dictionary,
                        coherence='c_v').get_coherence()

topics_dtm = dtm_model.dtm_coherence(time=2)
dtm_cs = CohereneceModel(topics=topics_dtm, texts=processing_data,
                        corpus=dtm_corpus, dictionary=dtm_dictionary,
                        coherence='c_v').get_coherence()

In [ ]:
# 결과 확인.
print("Coherence Score for LDA : %.3f" % lda_cs)
print("Coherence Score for DTM : %.3f" % dtm_cs)

- 강사님은 LDA : 0.774, DTM : 0.690으로 LDA가 더 높게 나왔음
- Coherence Score가 더 높게 나온 게 topic modeling이 더 잘된 것과는 별개, 정확히는 반대래
- topic modeling의 score가 높은 건 말 그대로 LDA 또는 DTM model이 얼마나 데이터들의 확률값을 수렴을 잘 했느냐의 지표
- 그것과 사람이 생각하는 topic modeling이 잘 된 건 차이가 뚜렷하지
    - 우리가 생각하는 키워드나 도메인 지식으로 봤을 때 이런 토픽들이 중요하겠구나라고 생각하는 게 다르니까
- LDA와 DTM에 들어있는 똑같은 2019년도 데이터라고 할지라도, LDA는 그냥 2019년도 데이터 하나를 딱 사용했지만 DTM은 2018년도부터 넘어온 파라미터에서 update된 것이기 때문에 coherence score가 2019년도 데이터에서만 영향을 받았다고 할 수 없는 것

- **제일 중요한 건 LDA와 DTM은 차이가 확실히 있다**라는 것을 알아야 해
- 특히 카카오톡 데이터같은 경우는 연도별 변화를 하기엔 애매
    - 전처리가 덜 돼서 의미가 그닥 없는 데이터,,
    - 좀 더 전처리를 해서 유의미한 단어들이 나온다고 하면, 그 단어들에 대해선 시간 변화를 주면 확실히 차이가 많이 나겠지 (ex. 논문 데이터)
- 그리고 카카오톡 데이터는 LDA와 DTM (topic modeling)을 했을 때 유의미한 데이터가 나온다고 물으면 애매해! 
    - 소셜 데이터고, 사용한 단체 카톡방 데이터는 공식적이기보단 사적인 대화가 오가다보니 양질의 데이터가 아니지
    - 슬랙같은 업무용 대화 데이터나 공식적, 스터디를 위해 모인 카톡방 데이터라면 확실히 차이가 많이 날 것임